# Deep Q-Learning for Lunar Landing

Please run in google colab

### Install required packages

In [ ]:
#!pip install gymnasium
#!pip install "gymnasium[atari, accept-rom-license]"
#!apt-get install -y swig
#!pip install gymnasium[box2d]
#!pip install ipython
#!pip install torch
#!pip install imageio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


### Import Libraries

In [ ]:
import random # For random action selection
import numpy as np # Handle arrays
import torch # PyTorch library
import torch.nn as nn # Define network layers
import torch.optim as optim # For optimizers like Adam
import torch.nn.functional as F # For activation functions like ReLU, mse_loss
from collections import deque # Used for the replay buffer (with maxlen) and tracking last 100 scores.
import gymnasium as gym # Interface with LunarLander-v3 environment
import imageio # Saves list of frames as .mp4.
import glob # Finds saved video files..
import io # Reads video for inline display.
import base64 # Encodes video for inline display.
from IPython.display import HTML # Displays the video in the notebook.

### Creating the Network class

In [ ]:
class Network(nn.Module):
    def __init__(self, state_size, action_size, seed=42):
        """
        🧠 THE AGENT’S BRAIN — A SIMPLE NEURAL NETWORK TO PREDICT ACTION VALUES (Q-VALUES)

        This network takes in the current state (e.g., position, velocity, angle, leg contact, etc.)
        and outputs a Q-value for each possible action — “how good is each action from this state?”

        Think of it like a scoring system:
          Input: “What’s happening right now?” → Output: “Score for doing nothing, left thrust, right thrust, main engine”

        Parameters:
          state_size  → number of inputs (how many numbers describe the environment)
          action_size → number of outputs (how many actions the agent can choose from)
          seed        → random seed for reproducibility (so your results are the same each time you train)
        """

        # Initialize parent class (nn.Module) — this gives us PyTorch neural network superpowers.
        super(Network, self).__init__()

        # 🎲 SET RANDOM SEED — so weight initialization is the same every time (helps with debugging and fair comparisons).
        self.seed = torch.manual_seed(seed)

        # 🧱 FIRST LAYER: Connects input state to 64 hidden neurons.
        # Example: LunarLander state_size=8 → this layer has 8 inputs, 64 outputs.
        self.fc1 = nn.Linear(state_size, 64)

        # 🧱 SECOND LAYER: 64 neurons → 64 neurons (keeps hidden representation rich).
        self.fc2 = nn.Linear(64, 64)

        # 🎯 OUTPUT LAYER: 64 neurons → action_size outputs (one Q-value per action).
        # Example: LunarLander action_size=4 → outputs 4 numbers: Q-values for each action.
        self.fc3 = nn.Linear(64, action_size)


    def forward(self, state):
        """
        🔄 FORWARD PASS — “THINKING” THROUGH THE NETWORK

        Given a state (or batch of states), compute the Q-values for all actions.

        Flow:
          state → [Layer 1 + ReLU] → [Layer 2 + ReLU] → [Layer 3] → Q-values

        ReLU (Rectified Linear Unit) = “If negative, make it zero. If positive, leave it.”
        → Helps network learn non-linear patterns without vanishing gradients.

        Returns:
          Tensor of shape (batch_size, action_size) — predicted Q-value for each action.
        """

        # ➡️ Pass state through first layer, then apply ReLU activation (introduces non-linearity).
        x = F.relu(self.fc1(state))

        # ➡️ Pass through second layer + ReLU.
        x = F.relu(self.fc2(x))

        # ➡️ Pass through final layer — NO activation (we want raw Q-values, positive or negative).
        return self.fc3(x)


### Training the AI

### Setting up the environment

In [ ]:
# Initialize the Lunar Lander environment in interactive rendering mode.
# 'LunarLander-v3' is the latest official version, featuring:
#   - Fixed non-determinism (resets wind/turbulence seed on each reset)
#   - Expanded observation bounds (x, y: ±2.5; velocities: ±10; angle: ±2π)
#   - Statistical independence between episodes
# render_mode="human" opens a window to visually observe the agent (ideal for final evaluation/demo).
env = gym.make('LunarLander-v3', render_mode="human")

# Extract observation space details.
# According to documentation, the state is an 8-dimensional vector containing:
#   [x, y, velocity_x, velocity_y, angle, angular_velocity, leg1_contact, leg2_contact]
state_shape = env.observation_space.shape  # Returns (8,) — the shape of the state vector.
state_size = env.observation_space.shape[0]  # Returns 8 — the number of state variables.

# Extract action space details.
# The environment has 4 discrete actions:
#   0: do nothing
#   1: fire left orientation engine
#   2: fire main engine
#   3: fire right orientation engine
number_actions = env.action_space.n  # Returns 4.

# Print environment dimensions for verification and debugging.
print('State shape: ', state_shape)      # Output: (8,)
print('State size: ', state_size)        # Output: 8
print('Number of actions: ', number_actions)  # Output: 4

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyper parameters

In [15]:
learning_rate = 5e-4 # best value taken after lot's of experimentation
minibatch_size = 100 # number of observations used in 1 step of training to update the model parameters
discount_factor = 0.99 # gamma parameter
# discount factor represents the present value of future rewards
# low discount value close to 0 (0.01) - makes agent short sighted
# high discount value close to 1 (0.99) - makes agent consider the future rewards in its accumulation of future rewards - giving best performance

replay_buffer_size = int(1e5) # replay buffer- stores number of experiences - including state, action reward, next state and whether done or not in the memory of agent
# purpose is to stabilize and improve training process
interpolation_parameter = 1e-3 # often denoted by the Greek letter tau (τ), that represents a time threshold or duration for considering a state-action pair as unvisited for a long time, influencing exploration - used in the sub update of target networks - best value after lot's of experimentation

### implementing experience replay

In [16]:
class ReplayMemory(object):
    def __init__(self, capacity):
        """
        🧠 MEMORY NOTEBOOK — STORES PAST EXPERIENCES FOR LATER LEARNING

        Think of this as the agent’s diary — it writes down everything it experiences
        (state, action, reward, next_state, done) and later flips back through pages
        to study random moments. This breaks the sequence of events and helps learning
        become more stable and less biased.

        Parameters:
          capacity → how many experiences it can remember before forgetting the oldest ones
        """

        # Set device (GPU if available, otherwise CPU) — we’ll convert stored data to tensors on this device later.
        # ⚠️ TYPO FIXED: was "cud:0" → corrected to "cuda:0"
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # Maximum number of experiences we can store — like the number of pages in the diary.
        self.capacity = capacity

        # The actual list where we store experiences — each experience is a tuple: (state, action, reward, next_state, done)
        self.memory = []


    def push(self, event):
        """
        📥 SAVE A NEW EXPERIENCE INTO MEMORY

        When the agent does something, we record what happened and store it.
        If memory is full, we forget the oldest experience (like a sliding window).
        """

        # Add the new experience (event) to the end of the memory list.
        self.memory.append(event)

        # If we’ve exceeded capacity, delete the oldest experience (the one at index 0).
        if len(self.memory) > self.capacity:
            del self.memory[0]


    def sample(self, batch_size):
        """
        🎲 RANDOMLY PICK A BATCH OF EXPERIENCES TO LEARN FROM

        Instead of learning in order (which can create bias), we randomly pick
        a bunch of past experiences — like shuffling flashcards before studying.
        Then we convert them into PyTorch tensors ready for the neural network.

        Returns:
          A tuple of tensors: (states, next_states, actions, rewards, dones)
          — all moved to the correct device (CPU or GPU)
        """

        # Randomly pick `batch_size` experiences from memory — this breaks correlation between consecutive steps.
        experiences = random.sample(self.memory, batch_size)

        # 🔽 CONVERT EACH PART OF THE EXPERIENCES INTO TENSORS 🔽

        # 📍 STATES: Stack all 'state' arrays vertically → shape (batch_size, state_size)
        # Convert to numpy → then to float tensor → send to device (GPU/CPU)
        states = torch.from_numpy(
            np.vstack([e[0] for e in experiences if e is not None])  # e[0] = state
        ).float().to(self.device)

        # 🎯 NEXT_STATES: Same as states, but for the state AFTER the action
        next_states = torch.from_numpy(
            np.vstack([e[3] for e in experiences if e is not None])  # e[3] = next_state
        ).float().to(self.device)

        # 🕹 ACTIONS: Stack action indices (usually integers) → convert to long tensor (required for .gather later)
        actions = torch.from_numpy(
            np.vstack([e[1] for e in experiences if e is not None])  # e[1] = action
        ).long().to(self.device)

        # 💰 REWARDS: Stack reward values → usually floats
        rewards = torch.from_numpy(
            np.vstack([e[2] for e in experiences if e is not None])  # e[2] = reward
        ).float().to(self.device)

        # 🚫 DONES: Stack “done” flags (True/False) → convert to 0/1 (uint8), then to float for math
        dones = torch.from_numpy(
            np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)  # e[4] = done
        ).float().to(self.device)

        # ✅ RETURN THE BATCH AS A TUPLE OF TENSORS — READY FOR .learn()!
        return (states, next_states, actions, rewards, dones)

### implementing the DQN class

🧠 KEY CONCEPTS:

- local_qnetwork → The “current brain” of the agent. It’s learning actively.
- target_qnetwork → A slightly older, more stable copy of the brain. Used to calculate targets so learning doesn’t go crazy.
- experiences → A batch of past memories: what state the agent was in, what it did, what reward it got, what state it ended up in, and whether the episode ended (dones).
- discount_factor → How much the agent cares about future rewards (e.g., 0.99 = cares a lot; 0.5 = only cares about immediate rewards).

In [17]:
class Agent():
    def __init__(self, state_size, action_size):
        """
        👶 INITIALIZATION — SETTING UP THE AGENT’S BRAIN AND TOOLS

        Think of this as the agent’s “birth” — we give it:
        - A brain (neural networks)
        - Memory (to store experiences)
        - Tools to learn (optimizer)
        - A counter to track when to learn

        Parameters:
          state_size → how many numbers describe the environment (e.g., position, velocity, angle, leg contact, etc.)
          action_size → how many possible actions the agent can take (e.g., 4 in LunarLander: do nothing, left, right, main engine)
        """

        # Store how big the state and action spaces are — the agent needs to know this to build its brain.
        self.state_size = state_size
        self.action_size = action_size

        # Decide whether to run on GPU (fast) or CPU (slow but works everywhere).
        # If you have a CUDA-compatible GPU, it’ll use that. Otherwise, fallback to CPU.
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # 🧠 LOCAL Q-NETWORK: The “student” brain — actively learning from experiences.
        # It predicts Q-values: “If I take this action in this state, how good will my future be?”
        self.local_qnetwork = Network(state_size, action_size).to(self.device)

        # 🧠 TARGET Q-NETWORK: The “teacher” brain — a stable version of the student.
        # Used to generate learning targets so the student doesn’t chase its own tail.
        self.target_qnetwork = Network(state_size, action_size).to(self.device)

        # 🧰 OPTIMIZER: The tool that adjusts the student brain’s weights to reduce prediction errors.
        # We use Adam — a popular, adaptive optimizer that works well for deep learning.
        self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)

        # 🧠 MEMORY: A “diary” of past experiences (state, action, reward, next_state, done).
        # We store them and later sample random batches to learn from — this breaks correlation and stabilizes learning.
        self.replayMemory = ReplayMemory(replay_buffer_size)

        # 🕐 STEP COUNTER: Tracks how many steps we’ve taken.
        # We’ll use this to decide when to pause and learn (e.g., every 4 steps).
        self.t_step = 0


    def step(self, state, action, reward, next_state, done):
        """
        📝 RECORD EXPERIENCE & DECIDE WHEN TO LEARN

        Every time the agent takes an action, we:
        - Save what happened (experience) into memory
        - Occasionally (e.g., every 4 steps), if we have enough memories, we sample a batch and learn from it
        """

        # Save this experience (state, action, reward, next_state, done) into the replay memory.
        self.replayMemory.push((state, action, reward, next_state, done))  # ← FIXED: was self.memory

        # Increment step counter, reset to 0 every 4 steps → triggers learning every 4 steps.
        self.t_step = (self.t_step + 1) % 4

        # If it’s time to learn (every 4 steps) AND we have enough memories to sample a batch...
        if self.t_step == 0:
            if len(self.replayMemory.memory) > minibatch_size:
                # Randomly sample a batch of past experiences.
                experiences = self.replayMemory.sample(minibatch_size)
                # Learn from them!
                self.learn(experiences, discount_factor)


    def act(self, state, epsilon=0.):
        """
        🎮 DECIDE WHAT ACTION TO TAKE RIGHT NOW

        Given the current state (what the agent sees), choose an action using:
        - EXPLOITATION: Pick the best-known action (according to current brain)
        - EXPLORATION: Sometimes pick a random action (to discover better strategies)

        Epsilon controls the balance:
          - epsilon = 1.0 → always random (full exploration)
          - epsilon = 0.0 → always pick best action (full exploitation)
          - epsilon = 0.1 → 10% random, 90% smart

        This is called “epsilon-greedy” strategy.
        """

        # Convert numpy state (from environment) into a PyTorch tensor, add batch dimension, and send to device (CPU/GPU).
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)

        # Temporarily switch the local Q-network to evaluation mode (disables dropout, batchnorm, etc. if used).
        self.local_qnetwork.eval()

        # Disable gradient calculation — we’re not learning right now, just predicting.
        with torch.no_grad():
            # Get Q-values for all possible actions in this state.
            action_values = self.local_qnetwork(state)

        # Switch the network back to training mode (in case we use layers like dropout or batchnorm).
        self.local_qnetwork.train()

        # 🎲 EPSILON-GREEDY DECISION:
        # If a random number is greater than epsilon → exploit (pick best action)
        if random.random() > epsilon:
            # Convert tensor to numpy, extract raw numbers, pick action with highest Q-value.
            return np.argmax(action_values.cpu().data.numpy())
        else:
            # Explore: pick a random action (helps discover new strategies).
            return random.choice(np.arange(self.action_size))


    def learn(self, experiences, discount_factor):
        # “Open up the batch of memories and separate them into their parts.”
        # states = what the lander saw before taking action (position, velocity, angle, etc.)
        # actions = what it did (fire left engine, right engine, main engine, or do nothing)
        # rewards = what it got for doing that (e.g., +100 for landing, -100 for crashing)
        # next_states = what the lander saw after taking the action
        # dones = whether the episode ended after that step (1 = yes, 0 = no)
        states, next_states, actions, rewards, dones = experiences

        # “Ask the target brain: In the next state, what’s the best possible action and its expected future reward?”
        # self.target_qnetwork(next_states) → Feed all the “next states” into the target network. It outputs Q-values for each possible action (e.g., 4 actions → 4 numbers per state).
        # .detach() → “Don’t track gradients here.” We’re using this for a target, not for learning right now.
        # .max(1)[0] → For each state, pick the maximum Q-value across all actions → this is the best expected future reward from that state.
        # .unsqueeze(1) → Add an extra dimension so the shape matches other tensors (e.g., turns shape (batch,) → (batch, 1)).
        # 👉 So Q_targets_next = best estimated future reward from the next state,
        # according to the stable target network.
        Q_targets_next = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)

        #  “Calculate what the total expected reward should be: immediate reward + discounted future reward — but only if the episode didn’t end.”
        # If the episode ended (dones = 1), then (1 - dones) = 0, so future reward is ignored → makes sense! No future after game over.
        # If not ended (dones = 0), then we add discount_factor * Q_targets_next → future reward matters.
        # 👉 This is the target Q-value — what we want our current network to
        # predict for the state-action pair.
        Q_targets = rewards + (discount_factor * Q_targets_next * (1 - dones))

        # “Ask the current brain: What did you predict the Q-value would be for the action we actually took?”
        # self.local_qnetwork(states) → current network predicts Q-values for all actions in each state.
        # .gather(1, actions) → picks out only the Q-value for the action that was actually taken in each experience.
        # 👉 This is the predicted Q-value — what the agent thought would
        # happen.
        Q_expected = self.local_qnetwork(states).gather(1, actions)

        # “Compare what the agent predicted vs. what it should have predicted. The bigger the difference, the worse it did.”
        # F.mse_loss = Mean Squared Error → average of (predicted - target)²
        # We want to minimize this → make predictions match targets.
        loss = F.mse_loss(Q_expected, Q_targets)

        # “Clear any old gradient info before calculating new ones.”
        # Gradients accumulate by default in PyTorch. We reset them to avoid mixing
        # up old and new learning signals.
        self.optimizer.zero_grad()

        # “Calculate how each weight in the network contributed to the error, so we know how to fix it.”
        # This is backpropagation — computes gradients of the loss with respect to all network parameters.
        loss.backward()

        # “Actually update the weights of the current brain to reduce the error.”
        # This takes a small step in the direction that reduces loss — i.e., makes predictions more accurate.
        self.optimizer.step()

        # “Gently nudge the target brain to be a little more like the current brain — but don’t copy it fully.”
        # soft_update usually looks like:
        #   target_weights = τ * local_weights + (1 - τ) * target_weights
        #   where τ (tau) is a small number like 0.001 → very slow updates.
        # 💡 Why? Because if the target network changed too fast, learning would
        # be unstable (like chasing a moving target). Soft updates keep it stable.
        self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

    def soft_update(self, local_model, target_model, interpolation_parameter):
      """
      interpolation_parameter = τ (tau)
      Soft update model parameters.
      θ_target = τ * θ_local + (1 - τ) * θ_target

      local_model: PyTorch model — weights will be copied from
      target_model: PyTorch model — weights will be copied to
      tau (float): interpolation parameter (e.g., 0.001)
      """
      for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
          # called Polyak averaging or exponential moving average, and it’s used all over deep RL (DQN, DDPG, SAC, etc.).
          #   tau = 0.001 → very slow update (used in DDPG, SAC)
          #   tau = 0.01 or 0.005 → moderate (common in DQN variants)
          #   tau = 1.0 → full copy (hard update — not recommended for stability)
          # .copy_: “Take the result of that weighted average and overwrite the target network’s weight with it — but in-place, without breaking the tensor connection.”
          # target_param = new_value
          # ❌ WRONG — breaks the model!
          #   it won’t update the actual network — it just reassigns the variable. You need to update the contents of the tensor, not the variable itself.
          # This line gently pulls the target network’s weights a tiny bit closer to the local network’s weights — keeping learning stable by avoiding sudden changes in the learning target.
          target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)


### Initializing the DQN network

In [18]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [ ]:
# Define training hyperparameters.
number_episodes = 2000  # Maximum number of episodes to train for.
maximum_number_timesteps_per_episode = 1000  # Max steps per episode (prevents infinite loops).
epsilon_starting_value = 1.0  # Start with full exploration (random actions).
epsilon_ending_value = 0.01  # Minimum exploration rate (nearly always exploit learned policy).
epsilon_decay_value = 0.995  # Decay rate per episode: ε = ε * 0.995 → slow, smooth decay.
epsilon = epsilon_starting_value  # Initialize epsilon.
# Track scores over the last 100 episodes to monitor if environment is "solved" (avg ≥ 200.0).
scores_on_100_episodes = deque(maxlen=100)

# Main training loop.
for episode in range(1, number_episodes + 1):
    # Reset environment at start of each episode.
    # Returns initial state and an info dict (ignored here with `_`).
    state, _ = env.reset()
    score = 0  # Cumulative reward for this episode.

    # Step through the environment for up to `maximum_number_timesteps_per_episode`.
    for t in range(maximum_number_timesteps_per_episode):
        # Select action using epsilon-greedy policy.
        # `agent.act(state, epsilon)` returns an integer in [0, 3]:
        #   0: do nothing
        #   1: fire left orientation engine
        #   2: fire main engine
        #   3: fire right orientation engine
        action = agent.act(state, epsilon)
        
        # Execute action in environment.
        # Returns:
        #   next_state: 8D vector [x, y, vx, vy, angle, angular_vel, leg1_contact, leg2_contact]
        #   reward:     Shaped reward (see below)
        #   done:       Boolean — True if episode terminated (crash, out-of-bounds, or asleep)
        #   _:          Truncated flag (for time limits — not used here)
        #   _:          Info dict (ignored)
        next_state, reward, done, _, _ = env.step(action)
        
        # Pass experience (S, A, R, S', done) to agent for learning.
        # This typically stores the transition in replay buffer and may trigger a learning update.
        agent.step(state, action, reward, next_state, done)
        
        # Update state for next iteration.
        state = next_state
        # Accumulate reward for this episode.
        # Reward components (per step):
        #   + Closer to pad (x=0, y=0) → higher reward
        #   + Slower velocity → higher reward
        #   + Less tilt (angle near 0) → higher reward
        #   + Each leg contact → +10 points
        #   - Firing side engine → -0.03 points/frame
        #   - Firing main engine → -0.3 points/frame
        #   +100 for safe landing, -100 for crash (applied on final step when done=True)
        score += reward
        
        # If episode terminated (crashed, landed, or out of bounds), exit inner loop.
        if done:
            break

    # Record episode score for averaging.
    scores_on_100_episodes.append(score)
    # Decay epsilon — reduce randomness over time to exploit learned policy.
    epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)

    # Print progress on same line (overwrites previous output).
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end="")

    # Every 100 episodes, print a full line (for logging/history).
    if episode % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))

    # Check if environment is "solved": average score over last 100 episodes ≥ 200.0.
    # According to documentation: "An episode is considered a solution if it scores at least 200 points."
    if np.mean(scores_on_100_episodes) >= 200.0:
        # Print success message: solved in the last 100 episodes.
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode-100, np.mean(scores_on_100_episodes)), end="")
        # Save model weights for later use or evaluation.
        torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
        # Exit training loop early.
        break

Episode 100	Average Score: -154.66
Episode 200	Average Score: -102.64
Episode 300	Average Score: -33.56
Episode 400	Average Score: -16.73
Episode 500	Average Score: 75.92
Episode 600	Average Score: 194.08
Episode 633	Average Score: 200.65
Environment solved in 533 episodes!	Average Score: 200.65

In [ ]:
def show_video_of_model(agent, env_name):
    """
    Records and saves a video of the trained agent interacting with the environment.
    
    According to the LunarLander-v3 documentation:
    - The agent receives an 8D state vector: [x, y, vx, vy, angle, angular_vel, leg1_contact, leg2_contact]
    - Actions are discrete: 0=do nothing, 1=left engine, 2=main engine, 3=right engine
    - Episode ends if lander crashes, flies out of bounds (|x| > 1), or "falls asleep" (stops moving)
    - Success is defined as average score >= 200 over 100 episodes
    """
    # Create environment in 'rgb_array' mode — returns frames as numpy arrays (no window popup).
    # This is ideal for generating videos programmatically.
    env = gym.make(env_name, render_mode='rgb_array')
    
    # Reset environment to initial state.
    # Returns initial state (8D vector) and info dict (ignored with `_`).
    state, _ = env.reset()
    
    # Flag to track episode termination.
    done = False
    
    # List to store rendered frames for video creation.
    frames = []
    
    # Step through environment until episode ends.
    while not done:
        # Capture current frame as numpy array (H x W x 3).
        frame = env.render()
        frames.append(frame)
        
        # Select action using trained agent (exploitation only — no epsilon-greedy).
        # `agent.act(state)` should return a tensor or integer in [0, 3].
        action = agent.act(state)
        
        # Execute action in environment.
        # Returns:
        #   next_state: 8D observation vector
        #   reward:     Shaped reward (distance, velocity, angle, contact, fuel cost)
        #   done:       True if episode terminated (crash, out-of-bounds, or asleep)
        #   _:          Truncated flag (time limit — not used in LunarLander)
        #   _:          Info dict (ignored)
        # Note: `.item()` converts tensor action to scalar if needed.
        state, reward, done, _, _ = env.step(action.item())
    
    # Close environment to free resources.
    env.close()
    
    # Save frames as MP4 video at 30 FPS.
    # File saved as 'video.mp4' in current directory.
    imageio.mimsave('video.mp4', frames, fps=30)

# Generate and save video of agent playing LunarLander-v3.
show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    """
    Displays the saved MP4 video inline in Jupyter/Colab notebook.
    Uses HTML5 video tag with base64 encoding for compatibility.
    """
    # Find all MP4 files in current directory.
    mp4list = glob.glob('*.mp4')
    
    if len(mp4list) > 0:
        # Take first MP4 file found.
        mp4 = mp4list[0]
        
        # Read video file as binary.
        video = io.open(mp4, 'r+b').read()
        
        # Encode as base64 string for embedding in HTML.
        encoded = base64.b64encode(video)
        
        # Display video with HTML5 player (autoplay, loop, controls, fixed height).
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        # Print error if no video found.
        print("Could not find video")

# Display the recorded video inline.
show_video()